In [1]:
import ray, random, os 
from ray.train.xgboost import XGBoostTrainer
from ray.air.config import ScalingConfig
import pandas as pd
ray.init(_temp_dir='/Volumes/SSD980/ray')
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from worker_standlone import WorkerStandAlone
from manager import Manager
from ray.train.rl import RLTrainer
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.tune.registry import register_env
from ray.tune import TuneConfig
from ray.tune.logger import pretty_print
from ray.tune.search.bayesopt import BayesOptSearch
from ray.air.config import RunConfig, CheckpointConfig
from ray.tune.schedulers import HyperBandScheduler
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

2023-09-28 10:33:58,989	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-09-28 10:34:05,632	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [3]:
if os.name == 'nt':
    path_to_save = "C:\\GitHub\\ray\\workertune"
else:
    path_to_save = "/Volumes/SSD980/ray/workertune1"


env = WorkerStandAlone()
def env_creator(env_config):
    return WorkerStandAlone()  

register_env("worker", env_creator)

asha_scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=5000,
    grace_period=200,
    reduction_factor=3,
    brackets=1,
)

param_space = {
    "env": "worker",
    "rollout_fragment_length": "auto",
    "framework": "tf2",
    "lr": tune.uniform(1e-5,1e-4),
    "gamma": tune.uniform(0.95, 0.9999),
    "lambda": tune.uniform(0.9,1.0),
    "entropy_coeff": tune.uniform(0.01,0.1),
    "vf_loss_coeff": tune.uniform(0.1,0.3),
    "num_workers": 2, 
    "log_level": "ERROR",
    "output": "logdir",
    "monitor": False,
}

analysis = tune.run(
    "A2C", 
    metric="episode_reward_mean", 
    num_samples=10,
    resume=False,
    mode="max",
    config=param_space, 
    local_dir=path_to_save,
    search_alg=None,
    scheduler=asha_scheduler,
    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=120),
    max_concurrent_trials=2,
    #checkpoint_config not checked yet
    checkpoint_config={
        "num_to_keep": 3,
        "checkpoint_score_attribute": "episode_reward_mean",
        "checkpoint_score_order": "max",
        "checkpoint_frequency": 10
    }
    )

print(pretty_print(analysis.last_result))
print("Best hyperparameters found were: ", pretty_print(analysis.best_config))

2023-09-28 10:34:20,290	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(


== Status ==
Current time: 2023-09-28 10:34:20 (running for 00:00:00.17)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 1800.000: None | Iter 600.000: None | Iter 200.000: None
Logical resource usage: 0/8 CPUs, 0/0 GPUs
Result logdir: /Volumes/SSD980/ray/workertune/A2C
Number of trials: 2/10 (2 PENDING)




(pid=16177) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=16177) 2023-09-28 10:34:26,675	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=16177) 2023-09-28 10:34:26,675	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=16177) 2023-09-28 10:34:26,675	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=16199) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future! [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#lo

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
A2C_worker_d1b9a_00000,449120,"{'ObsPreprocessorConnector_ms': 0.025473322187151228, 'StateBufferConnector_ms': 0.004147802080426898, 'ViewRequirementAgentConnector_ms': 0.09557826178414482}","{'num_env_steps_sampled': 449120, 'num_env_steps_trained': 449120, 'num_agent_steps_sampled': 449120, 'num_agent_steps_trained': 449120}",{},6305,{},199692,67930.1,-2,1,"{'learner': {'default_policy': {'learner_stats': {'cur_lr': 7.899221236584708e-05, 'entropy_coeff': 0.013264811597764492, 'policy_loss': -25250966000000.0, 'policy_entropy': -547.2268, 'var_gnorm': 24.321335, 'vf_loss': 78.606}, 'grad_gnorm': 40.000004, 'vf_explained_var': 2.0623207e-05, 'custom_metrics': {}, 'num_agent_steps_trained': 32, 'num_grad_updates_lifetime': 14035, 'diff_num_grad_updates_vs_sampler_policy': 14034}}, 'num_env_steps_sampled': 449120, 'num_env_steps_trained': 449120, 'num_agent_steps_sampled': 449120, 'num_agent_steps_trained': 449120}",449120,449120,449120,7040,703.566,449120,7040,703.566,0,3,0,0,7040,"{'cpu_util_percent': 90.04285714285716, 'ram_util_percent': 84.07142857142857}",{},{},{},"{'mean_raw_obs_processing_ms': 0.4966412519011265, 'mean_inference_ms': 1.313350577425996, 'mean_action_processing_ms': 0.18979633291464332, 'mean_env_wait_ms': 0.30858048058786863, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 199691.5709291429, 'episode_reward_min': -2.0, 'episode_reward_mean': 67930.08215603894, 'episode_len_mean': 6305.0, 'episode_media': {}, 'episodes_this_iter': 1, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [28632.2039503423, 15821.731639282312, 28435.84409679193, 136440.63439996625, 43397.19055744865, 16837.671950103486, 95882.31007636047, 100696.85642381455, 104942.43049612321, 72280.29872748531, 85743.60329255226, 114369.59861984605, 144998.41999530263, 68588.85607545768, 127916.77841141628, 97041.3447894823, 83036.82732863598, 123541.65700333091, 80784.35932328727, 78039.66454649041, 199691.5709291429, 144208.99659605042, 132966.29332317575, 110423.7524998668, 110558.89360707719, 136515.46768895653, 119440.54595050236, 118951.51611160603, 97767.02143994904, 168319.73085720837, 169132.70989083825, 185843.3307816854, 63501.38055788231, 64518.550439755985, 70295.53417521623, 69853.88897908572, 69951.14948620647, 107033.62312728167, 144367.1137345843, 133891.82607493177, 116176.25533165783, 63849.324734687805, 64332.88672590256, 100868.15902352333, 56845.12265641987, 58929.13983607292, 2095.9199731200933, 6692.6064175367355, 5344.061789631844, 39598.08112300187, 33006.46030293405, 3218.220173448324, 5661.753787561087, 36489.44458684325, 37810.86443160474, 3422.540355358273, 7909.586325287819, 48205.14539361, 0.0, -2.0, -2.0, -1.0, -1.0, 0.0, -1.0, -2.0, 0.0, -1.0, -1.0, 0.0], 'episode_lengths': [6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305, 6305]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.4966412519011265, 'mean_inference_ms': 1.313350577425996, '

...

(RolloutWorker pid=16203) =========HRL is done============= [repeated 4x across cluster]
(RolloutWorker pid=16203) Worker is done [repeated 4x across cluster]
(RolloutWorker pid=16203) day: 6304, episode: 20 [repeated 4x across cluster]
(RolloutWorker pid=16203) Beginn_Portfolio_Value: 1000000.0 [repeated 4x across cluster]
(RolloutWorker pid=16203) End Total Assets: 5804685.5 [repeated 4x across cluster]
(RolloutWorker pid=16203) Total PnL: 4804686.0 [repeated 4x across cluster]
(RolloutWorker pid=16203) ================================= [repeated 4x across cluster]


(RolloutWorker pid=16199) 2023-09-28 10:41:51,271	DEBUG json_writer.py:81 -- Wrote 7218 bytes to <_io.TextIOWrapper name='/Volumes/SSD980/ray/workertune/A2C/A2C_worker_d1b9a_00000_0_entropy_coeff=0.0133,gamma=0.9673,lambda=0.9543,lr=0.0001,vf_loss_coeff=0.2674_2023-09-28_10-34-20/output-2023-09-28_10-40-34_worker-1_1.json' mode='w' encoding='UTF-8'> in 0.00028705596923828125s [repeated 762x across cluster]
(RolloutWorker pid=16199) 2023-09-28 10:41:56,295	DEBUG json_writer.py:81 -- Wrote 6506 bytes to <_io.TextIOWrapper name='/Volumes/SSD980/ray/workertune/A2C/A2C_worker_d1b9a_00000_0_entropy_coeff=0.0133,gamma=0.9673,lambda=0.9543,lr=0.0001,vf_loss_coeff=0.2674_2023-09-28_10-34-20/output-2023-09-28_10-40-34_worker-1_1.json' mode='w' encoding='UTF-8'> in 0.00033402442932128906s [repeated 680x across cluster]
(RolloutWorker pid=16199) 2023-09-28 10:42:01,349	DEBUG json_writer.py:81 -- Wrote 6588 bytes to <_io.TextIOWrapper name='/Volumes/SSD980/ray/workertune/A2C/A2C_worker_d1b9a_00000_

== Status ==
Current time: 2023-09-28 10:42:20 (running for 00:08:00.32)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 1800.000: None | Iter 600.000: None | Iter 200.000: None
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
Current best trial: d1b9a_00001 with episode_reward_mean=93283.18782236197 and parameters={'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': True, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_o

(RolloutWorker pid=16199) 2023-09-28 10:42:21,649	DEBUG json_writer.py:81 -- Wrote 6631 bytes to <_io.TextIOWrapper name='/Volumes/SSD980/ray/workertune/A2C/A2C_worker_d1b9a_00000_0_entropy_coeff=0.0133,gamma=0.9673,lambda=0.9543,lr=0.0001,vf_loss_coeff=0.2674_2023-09-28_10-34-20/output-2023-09-28_10-40-34_worker-1_1.json' mode='w' encoding='UTF-8'> in 0.0002932548522949219s [repeated 797x across cluster]
(RolloutWorker pid=16199) 2023-09-28 10:42:26,749	DEBUG json_writer.py:81 -- Wrote 6523 bytes to <_io.TextIOWrapper name='/Volumes/SSD980/ray/workertune/A2C/A2C_worker_d1b9a_00000_0_entropy_coeff=0.0133,gamma=0.9673,lambda=0.9543,lr=0.0001,vf_loss_coeff=0.2674_2023-09-28_10-34-20/output-2023-09-28_10-40-34_worker-1_1.json' mode='w' encoding='UTF-8'> in 0.0002608299255371094s [repeated 775x across cluster]
(RolloutWorker pid=16199) 2023-09-28 10:42:31,789	DEBUG json_writer.py:81 -- Wrote 6648 bytes to <_io.TextIOWrapper name='/Volumes/SSD980/ray/workertune/A2C/A2C_worker_d1b9a_00000_0_

(RolloutWorker pid=16199) 2023-09-28 10:43:47,622	DEBUG json_writer.py:81 -- Wrote 6581 bytes to <_io.TextIOWrapper name='/Volumes/SSD980/ray/workertune/A2C/A2C_worker_d1b9a_00000_0_entropy_coeff=0.0133,gamma=0.9673,lambda=0.9543,lr=0.0001,vf_loss_coeff=0.2674_2023-09-28_10-34-20/output-2023-09-28_10-40-34_worker-1_1.json' mode='w' encoding='UTF-8'> in 0.0007741451263427734s [repeated 507x across cluster]


In [ ]:
env = WorkerStandAlone()
def env_creator(env_config):
    return WorkerStandAlone()  

register_env("worker", env_creator)



analysis = tune.run(
        "A2C",
        metric="episode_reward_mean",
        mode="max",
        config={
            "env": "worker",
            "env_config": {"initial_capital": 1e6},
            "lr": tune.uniform(1e-5, 1e-4),
            "train_batch_size": tune.choice([10000, 20000, 40000]),
}, )

In [4]:


n_iterations = 30000


env = WorkerStandAlone(data=data)

state = env.reset()

for _ in range(n_iterations):
    action = env.action_space.sample()  # Generate random actions for the manager
    worker_dones = {...}  # You need to provide the values for worker_dones
    worker_truncateds = {...}  # You need to provide the values for worker_truncateds

    obs, reward, done, truncated, info = env.step(action)

    if done:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

=========HRL is done=============
Worker is done
day: 6304, episode: 1
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 2301572.0
Total PnL: 1301572.875
Episode finished!
=========HRL is done=============
Worker is done
day: 6304, episode: 2
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 5299982.5
Total PnL: 4299987.0
Episode finished!
=========HRL is done=============
Worker is done
day: 6304, episode: 3
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 1822478.875
Total PnL: 822476.375
Episode finished!
=========HRL is done=============
Worker is done
day: 6304, episode: 4
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 1857416.875
Total PnL: 857417.625
Episode finished!


In [2]:
#Mit vielen policies
import sys
# Redirect the standard error to the log file

# Create a log file
log_file = open("console_logs.txt", "w")


# Redirect the standard output to the log file
sys.stdout = log_file


env = HRL()
def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def create_policy_spec(worker_id):
    # print(f"Creating policy for {worker_id} with obs space {env.observation_space[worker_id]} and action space {env.action_space[worker_id]}")
    return PolicySpec(
        observation_space=env.observation_space[worker_id],
        action_space=env.action_space[worker_id],
        config={}
    )

manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "manager_policy": manager_policy_spec,
}

for worker_id in env.workers:
    policies[worker_id] = create_policy_spec(worker_id)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id == 'manager':
        # print(f"!!!!!! policy mapping manager: {agent_id}")
        return "manager_policy"
    elif agent_id in env.workers:
        return agent_id
    else:
        print("defaul policy triggered")
        return "default_policy"

param_space = {
     "env": "hrl",
    "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "lr": tune.uniform(1e-5,1e-4),
        "gamma": tune.uniform(0.95, 0.9999),
        "lambda": tune.uniform(0.9,1.0),
        "entropy_coeff": tune.uniform(0.01,0.1),
        "vf_loss_coeff": tune.uniform(0.1,0.3),
        "num_workers": 3, 
        #Change for Debugging
        "log_level": "DEBUG",
        "output": "logdir",
        "monitor": True,
}

analysis = tune.run(
    "A2C",
    metric="episode_reward_mean", 
    num_samples=5,
    mode="max",
    config=param_space, 
    local_dir="/Volumes/SSD980/ray/results/tunerun2",
    search_alg=None,
    scheduler=None,
    progress_reporter=CLIReporter(max_progress_rows=5,max_report_frequency=120),
    max_concurrent_trials=2,
    #checkpoint_config not checked yet
    checkpoint_config={
        "num_to_keep": 1,
        "checkpoint_score_attribute": "episode_reward_mean",
        "checkpoint_score_order": "max",
        "checkpoint_frequency": 10
    }
)

2023-09-28 08:39:25,055	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
2023-09-28 08:39:25,087	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-09-28 08:39:25,090	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023-09-28 08:39:25,120	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
A2C_hrl_c3ec2_00000,8064,{},"{'num_env_steps_sampled': 576, 'num_env_steps_trained': 576, 'num_agent_steps_sampled': 8064, 'num_agent_steps_trained': 8064}",{},nan,{},nan,nan,nan,0,8064,8064,576,544,53.4054,576,544,53.4054,0,3,0,0,544,"{'cpu_util_percent': 83.28571428571426, 'ram_util_percent': 84.42857142857143}",{},{},{},{},"{'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}}","{'training_iteration_time_ms': 648.727, 'sample_time_ms': 570.327, 'learn_time_ms': 60.712, 'learn_throughput': 527.075, 'synch_weights_time_ms': 17.043}"
A2C_hrl_c3ec2_00001,8064,{},"{'num_env_steps_sampled': 576, 'num_env_steps_trained': 576, 'num_agent_steps_sampled': 8064, 'num_agent_steps_trained': 8064}",{},nan,{},nan,nan,nan,0,8064,8064,576,544,52.2368,576,544,52.2368,0,3,0,0,544,"{'cpu_util_percent': 76.5, 'ram_util_percent': 84.32666666666665}",{},{},{},{},"{'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}}","{'training_iteration_time_ms': 661.05, 'sample_time_ms': 567.554, 'learn_time_ms': 75.07, 'learn_throughput': 426.27, 'synch_weights_time_ms': 17.809}"


(A2C pid=5199) WARNING:tensorflow:6 out of the last 6 calls to <bound method EagerTFPolicyV2._learn_on_batch_helper of A3CTF2Policy_traced> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details. [repeated 6x across cluster]
(A2C pid=5199) 2023-09-28 08:39:55,220	DEBUG rollout_worker.py:825 -- Training out:
(A2C pid=5199) { 'ABT.US': { 'custom_metrics': {},
(A2C pid=5199)                       'diff_num_grad_updates_vs_sampler_policy': 0, [repeated 14x across cluster]
(A2C pid=5199)  

In [ ]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
initial_params = [{
    "lr": 0.001,
    "gamma": 0.92,
    "lambda": 0.95,
    "entropy_coeff": 1e-3,
    "vf_loss_coeff": 0.5,
    "model": {
        "fcnet_hiddens": 64,
        "fcnet_activation":"relu",
    },
}]

search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "gamma": tune.uniform(0.9, 0.99),
    "lambda": tune.uniform(0.9, 1.0),
    "entropy_coeff": tune.loguniform(1e-4, 1e-1),
    "vf_loss_coeff": tune.uniform(0.1, 1.0),
    # "model": {
    #     "fcnet_hiddens": tune.grid_search([[64, 64], [128, 128], [256, 256]]),
    #     "fcnet_activation": tune.choice(["relu", "tanh"]),
    # },
}


algo = HyperOptSearch(space=search_space,metric="episode_reward_mean", mode="max",)


param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },

        "num_workers": 1,  
        "num_cpus_per_trial": 1,
         "lr": tune.loguniform(1e-4, 1e-1),
    "gamma": tune.uniform(0.9, 0.99),
    "lambda": tune.uniform(0.9, 1.0),
    "entropy_coeff": tune.loguniform(1e-4, 1e-1),
    "vf_loss_coeff": tune.uniform(0.1, 1.0),
}



param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
          "lambda": tune.uniform(0.95, 1.0),
          "vf_loss_coeff": tune.uniform(0.1, 1.0),
          "entropy_coeff": tune.uniform(1e-4, 1e-1),
        "num_workers": 1,  
        "num_envs_per_worker": 1
}

analysis = tune.run("A2C", 
                    metric="episode_reward_mean", 
                    mode="max",
                    config=param_space,
                    num_samples=10,
                    stop={"training_iteration": 100},
                    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=600),
                    # local_dir="/Users/floriankockler/rayresults/overnight1",
                    storage_path="/Users/floriankockler/Documents/GitHub.nosync/raystorage",
                    checkpoint_config=CheckpointConfig(
                        num_to_keep=2,
                        checkpoint_score_attribute="episode_reward_mean", 
                        checkpoint_score_order="max"
                        )
                    )


best = analysis.best_trial
print(pretty_print(best.last_result))

In [ ]:

env = HRL()

n_iterations = 1

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done["__all__"]:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 



policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}

    
algo = PPOConfig().environment(env=HRL).multi_agent(
        policies=policies,
        policy_mapping_fn = policy_mapping_fn,  
    ).training(train_batch_size=4000).build()

print(algo.train())

In [ ]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

analysis = tune.run("A2C", metric="episode_reward_mean", mode="max",config=param_space)

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
algo = PPOConfig().environment(env=HRL).multi_agent(
    policies={
        "policy_1": ()
    }
)

In [ ]:
from ray.tune import CLIReporter
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
env = HRL()

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune_config,
    param_space=param_space,

    run_config=run_config,
)
results = tuner.fit()


In [ ]:
import random
import os
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hrl", env_creator)

manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }
env = HRL(hrl_config)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 


first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}


def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    #"rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(200, 1500),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),

}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env_config": hrl_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 1,  # 1 for training + 4 for sampling
        "num_cpus_per_trial": 3,
        # "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        # "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/autobatch", progress_reporter=reporter),
)
results = tuner.fit()